In [1]:
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

# ConversationKGMemory

In [2]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationKGMemory

In [3]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

memory = ConversationKGMemory(llm=llm, return_messages=True)
memory.save_context(
    {"input": "이쪽은 Pangyo에 거주중인 김셜리씨 입니다."},
    {"output": "김셜리씨는 누구시죠?"},
)
memory.save_context(
    {"input": "김셜리씨는 우리 회사의 신입 디자이너입니다."},
    {"output": "만나서 반갑습니다."}
)

In [7]:
memory.load_memory_variables({"input": "김셜리씨는 누구?"})

{'history': [SystemMessage(content='On 김셜리씨: 김셜리씨 is 신입 디자이너. 김셜리씨 is in 우리 회사.', additional_kwargs={}, response_metadata={})]}

# Chain 에 메모리 활용하기

In [9]:
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import ConversationChain

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

template = """The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.
The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""

prompt = PromptTemplate(
    template=template,
    input_variables=["input", "history"]
)

conversation_with_kg = ConversationChain(
    llm=llm, prompt=prompt, memory=ConversationKGMemory(llm=llm, return_messages=True)
)

C:\Users\kakao\AppData\Local\Temp\ipykernel_19860\1917994803.py:24: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation_with_kg = ConversationChain(


In [10]:
conversation_with_kg.predict(
    input="My name is Teddy. Shirley is a coworker of mine, and she`s a new designer at our comany."
)

"Hi Teddy! It's great to meet you. How's it going with Shirley, the new designer? Have you had a chance to work together on any projects yet?"

In [11]:
conversation_with_kg.memory.load_memory_variables({"input": "who is Shirley"})

{'history': [SystemMessage(content='On Shirley: Shirley is a coworker. Shirley is a new designer. Shirley works at our company.', additional_kwargs={}, response_metadata={})]}